## 1x1 convolution 

GoogLeNet[Inception network]에서는 1x1 convolution을 통해 필터의 수를 조절함으로써 연산량을 줄였다.

먼저 1*1 Convolution을 사용하면 필터의 개수가 몇 개 인지에 따라 output의 dimension은 달라지지만, 원래 가로 세로의 사이즈는 그대로 유지된다.

그래서 filter 의 개수를 원래 input의 dimension 보다 작게 하면, dimension reduction의 효과가 난다.
Dimension reduction을 이용하기 위해 1*1 conv 의 개수를 줄이면, activation의 depth가 작아져서 filter의 총 parameter의 개수가 감소한다.

이 형태를 bottleneck 구조라고 하는데, dimension reduction을 한 뒤 모든 연산을 하고 다시 filter의 갯수를 늘려서 고차원으로 늘리는 방법을 이용하기 때문에 bottleneck이라고 부른다.

1x1 convolution 이면 사실 convolution의 의미는 사라지는 것 같다. 하지만, 딥러닝에서 1x1 convolution 을 사용할 때가 있다.
그것은 바로 filter의 수를 조절하는 것이다.

(1x1 convolution 이라고만 말하지 않고, filter 수(차원의 수, 채널의 수와 혼용된다.)라는 것이 함께 말해야 의미가 있다.)

입력하는 채널의 수와 출력하는 채널의 수가 완전히 동일하다면 convolution의 큰 의미는 없을 것으로 생각된다. 단순히 projection만 하는 것이니까. 하지만 차원의 수를 바꿔준다면! 이야기는 달라진다. 차원을 크게도, 그리고 작게도 만든다. 입력받는 것에 비해 적은 수의 차원으로 채널의 수로, filter의 수로 만들어 준다면- 차원이 축소된 정보로 연산량을 크게 줄일 수 있다. 한번 이렇게 줄여 놓으면 뒤로가면 연계되는 연산량의 수, 파라미터의 수가 확 줄어들기 때문에 같은 컴퓨팅 자원과 시간 자원으로 더 깊은 네트워크를 설계하고 학습할 수 있게 된다. 그런데, 차원을 단순히/무작정 작게만 만든다고 다 되는 것은 아니다. 적당한 크기가 필요하고, 그 다음의 레이어에서 학습할 만큼은 남겨둔 적당한 차원이어야한다. 이러한 구조를 잘 활용한 것이 bottleneck 이라는 구조이다.

bottleneck은 1x1 convolution으로 좁아졌다가, 3x3 convolution(원래 하고자 했던 연산)을 하고, 1x1 convolution으로 다시 차원의 깊이도 깊게만드는 block을 bottleneck 이라고 부른다. 실제로 하는 일은 비용이 많이 드는 3x3, 5x5 convolution을 하기 전에, 1x1 convolution을 하여 차원을 감소시킨 후에 3x3, 5x5 convolution을 수행하는 것이다. inception 모델에서 활영되어 파라미터의 수는 줄이고 레이어를 깊게 하는데 활용되었으며, 특히 ResNet에서 엄청난 효과를 나타내었다.

### 계산량 감소
<img src='./imgs/cnn1.png'>

채널 수 조절은 곧 계산량 감소로 이어지게 되어 우리가 네트워크를 구성할 때 좀 더 깊어질 수 있도록 도움을 줍니다.

파라미터의 수가 많으면 아무리 GPU가 좋아도 감당하기 힘들테니까요.

먼저 결과부터 볼까요? 위의 그림에서 윗 부분과 아랫 부분의 차이는 1x1 컨볼루션을 거치느냐, 거치지 않느냐입니다.

빨간색 글씨는 해당 과정에서 사용되는 전체 파라미터 개수를 의미합니다. 윗 부분은 160백만개의 파라미터 수가 필요하고, 아랫 부분은 약 44백만개의 수가 필요하네요. 대략 4배의 차이입니다. 실제로 모델 전체에 이를 사용해서 직접 비교하면 수십 배의 차이가 남을 알 수 있습니다.

파란색 글씨는 컨볼루션을 거치고 난 후의 결과값에서 사용되는 파라미터 수이며,

초록색 글씨는 컨볼루션 과정에서 사용되는 필터가 가지는 파라미터의 수입니다.

각 숫자가 나타내는 의미가 무엇인지 생각해보면서, 해당 파라미터를 직접 계산해보는 것을 추천드립니다. 아직 컨볼루션의 개념이 확립되지 않았다면, 이해하기까지 길지는 않지만 어느 정도의 시간이 소요될 것입니다. 

### 삽화를 통해보다 직관적이고 간결하게 

입력이 (n_H, n_W, n_c_prev)라고 가정 해 봅시다.  (1 * 1 * n_c_prev)으로의 단일 신경 세포 (완전히 연결된 네트워크 - 즉 왜 N / N / w에 w ) 즉 그들을 곱 입력에 (n_c_prev) 숫자 복용 (1 * 1 * n_c_prev )를 추가 한 다음 ReLu & 출력 (n_H, n_W)을 취하고 여러 필터 (n_c)가 있으면 출력은 (n_H, n_W, n_C)가됩니다.

따라서 풀링 레이어를 사용하여 공간 크기 (n_H, n_W)를 줄이고 1 * 1 전환을 사용하여 n_c_prev (즉, 채널 수)를 줄여 많은 계산을 절약 할 수 있습니다. 예를 들어

<img src='./imgs/cnn2.png'>

따라서,

* 1x1 컨 벌루 셔널 레이어를 사용하여 n_C를 줄일 수 있지만 n_H, n_W는 줄일 수 없습니다.

* 풀링 계층을 사용하여 n_H, n_W 및 n_C를 줄일 수 있습니다.


<img src='./imgs/cnn3.png'>

다시 말해,

* 1 * 1 CONV 필터를 사용하여 무엇을합니까? 당신은 기본적으로 (elementwise는 * + 뒤에) "W 크기의 이미지 / featureMap 이상이 볼륨"1 * 1 * num_input_channel_of_featureMap "크기 가중치 및 말다을 H num_input_channel_of_featureMap가"과 무엇을 얻는 것은 크기 "W의 *의 H"의 출력이다.

* 이제 이런 종류의 "1 * 1 * num_input_channel_of_featureMap"의 "#filters"번호를 사용 하고 최종 출력으로 "W H #filter" 볼륨을 얻을 수 있습니다 !

* 더 정확하게는, 입력 피쳐 맵에서 동일한 위치에 32 개의 다른 가중치 (1 * 1 볼륨)와 "1 * 1"을 곱한 후 ReLu가 뒤 따르고 녹색 픽셀로 표시되는 해당 숫자 하나를 얻는 것입니다. !

* 그러나 물론 채널 수를 입력 기능 맵과 동일하게 유지하려면 1 * 1 CONV 가하는 한 가지 간단한 방법은 ReLU 비선형 성을 적용하여 복잡한 기능을 배울 수 있다는 것입니다. NN .

### 비선형성

Inception Version을 보시면 여러개의 1x1 conv로 구성되어 있습니다. 이때 Relu를 계속해서 사용하게 되면 비선형성이 증가하게 됩니다. Relu의 사용목적은 네트워크에 비선형성을 더해주기 위함이니까요.

비선형성이 증가한다는 것은 그만큼 복잡한 패턴을 좀 더 잘 인식할 수 있게 된다는 의미와 비슷하겠죠?

 